In [23]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point
import osmnx as ox

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [24]:
nodes = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Vias_Transporte/nodes_AMG.geojson', driver='GeoJSON')
print(nodes.shape)
nodes.head(2)

(172090, 5)


,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3.0,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,POINT (-103.31665 20.70065)


In [25]:
edges = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Vias_Transporte/edges_speed_AMG.geojson', driver='GeoJSON')
print(edges.shape)
edges.head(2)

(414773, 16)


,id,u,v,key,osmid,oneway,length,grade,grade_abs,bridge,junction,service,tunnel,walkspeed,time_min,geometry
0,-1.922695e+09,1.680913e+09,1.680913e+09,0.0,155732288,False,45.603,0.022,0.022,None,None,None,None,3.703559,0.738797,"LINESTRING (-103.25834 20.68905, -103.25844 20..."
1,-1.922105e+09,4.343498e+09,4.343498e+09,0.0,436461417,False,141.661,0.042,0.042,None,None,None,None,3.453176,2.461404,"LINESTRING (-103.19724 20.66496, -103.19742 20..."


In [26]:
edges = edges[~edges.u.isna()].copy()
print(edges.shape)
edges.head(2)

(414772, 16)


,id,u,v,key,osmid,oneway,length,grade,grade_abs,bridge,junction,service,tunnel,walkspeed,time_min,geometry
0,-1.922695e+09,1.680913e+09,1.680913e+09,0.0,155732288,False,45.603,0.022,0.022,None,None,None,None,3.703559,0.738797,"LINESTRING (-103.25834 20.68905, -103.25844 20..."
1,-1.922105e+09,4.343498e+09,4.343498e+09,0.0,436461417,False,141.661,0.042,0.042,None,None,None,None,3.453176,2.461404,"LINESTRING (-103.19724 20.66496, -103.19742 20..."


In [27]:
museo = nodes.loc[nodes.osmid==999999].copy()

In [28]:
nodes_analysis = nodes.set_index('osmid')
edges_analysis = edges.set_index(['u','v','key'])

In [29]:
nodes_dist = aup.calculate_distance_nearest_poi(museo, nodes_analysis, 
                edges_analysis, 'museo', 'osmid', wght='time_min')

In [30]:
nodes_dist.rename(columns={'dist_museo':'time_museo'}, inplace=True)

In [31]:
nodes_dist.to_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Analisis/nodes_time_AMG_NewConect.geojson', driver='GeoJSON')

In [32]:
hex_gdf = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Area_Estudio/Hex_AMG.geojson', driver='GeoJSON')
print(hex_gdf.shape)
hex_gdf.head(2)

(5205, 3)


,hex_id_8,CVEGEO,geometry
0,88498cd6b5fffff,14001,"POLYGON ((-102.89959 20.75324, -102.89907 20.7..."
1,88498c99edfffff,14001,"POLYGON ((-102.94697 20.69376, -102.94243 20.6..."


In [33]:
nodes_dist.reset_index(inplace=True)
nodes_dist = nodes_dist.set_crs("EPSG:4326")
hex_gdf = hex_gdf.set_crs("EPSG:4326")
col_name = 'time_museo'
hex_dist = aup.group_by_hex_mean(nodes_dist, hex_gdf, 8, col_name)

In [34]:
hex_gdf = hex_gdf.merge(hex_dist[['hex_id_8',col_name]], 
on='hex_id_8')

In [35]:
hex_gdf

,hex_id_8,CVEGEO,geometry,time_museo
0,88498cd6b5fffff,14001,"POLYGON ((-102.89959 20.75324, -102.89907 20.7...",0.000000
1,88498c99edfffff,14001,"POLYGON ((-102.94697 20.69376, -102.94243 20.6...",0.000000
2,88498c993bfffff,14001,"POLYGON ((-102.94697 20.69376, -102.94645 20.6...",0.000000
3,88498c8a07fffff,14001,"POLYGON ((-103.00449 20.78276, -103.00956 20.7...",0.000000
4,88498cd6edfffff,14001,"POLYGON ((-102.88529 20.70739, -102.88477 20.7...",0.000000
...,...,...,...,...
5200,88498c98d5fffff,14124,"POLYGON ((-103.05428 20.69941, -103.04974 20.7...",783.320079
5201,88498c815bfffff,14124,"POLYGON ((-103.11272 20.74897, -103.10817 20.7...",701.173334
5202,8849aa6e89fffff,14124,"POLYGON ((-102.93286 20.55354, -102.93234 20.5...",982.078083
5203,88498c9901fffff,14124,"POLYGON ((-102.93423 20.66349, -102.92968 20.6...",975.653643


In [36]:
hex_gdf.loc[hex_gdf.time_museo>0].to_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Analisis/hex_time_AMG.geojson', driver='GeoJSON')

In [37]:
mun_gdf = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Limites_Politicos/AMG_Jalisco12_UTM_v1.shp', driver='ESRIShapefile')

In [38]:
mun_gdf = mun_gdf.to_crs("EPSG:4326")

In [39]:
nodes = gpd.read_file('/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Analisis/nodes_time_AMG_NewConect.geojson', driver='GeoJSON')
print(nodes.shape)
nodes.head(2)

(171941, 6)


,osmid,x,y,street_count,time_museo,geometry
0,28751344,-103.306309,20.711533,3.0,145.197878,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,140.750865,POINT (-103.31665 20.70065)


In [40]:
res = 9
hex_gdf = aup.create_hexgrid(mun_gdf, res)

In [41]:
nodes_dist = nodes.copy()
#nodes_dist.reset_index(inplace=True)
nodes_dist = nodes_dist.set_crs("EPSG:4326")
hex_gdf = hex_gdf.set_crs("EPSG:4326")
col_name = 'time_museo'
hex_dist = aup.group_by_hex_mean(nodes_dist, hex_gdf, res, col_name)

In [42]:
hex_gdf = hex_gdf.merge(hex_dist[[f'hex_id_{res}',col_name]], 
on=f'hex_id_{res}')

In [43]:
hex_gdf

,hex_id_9,geometry,time_museo
0,89498c95523ffff,"POLYGON ((-103.27106 20.69008, -103.26984 20.6...",221.124577
1,89498c9732fffff,"POLYGON ((-103.27311 20.66965, -103.27189 20.6...",244.371072
2,89498c969d7ffff,"POLYGON ((-103.39540 20.68491, -103.39418 20.6...",107.712977
3,89498c96e93ffff,"POLYGON ((-103.36033 20.65455, -103.35911 20.6...",167.156440
4,89498c95cdbffff,"POLYGON ((-103.27531 20.71104, -103.27409 20.7...",0.000000
...,...,...,...
26178,89498c911a7ffff,"POLYGON ((-103.12986 20.61135, -103.12864 20.6...",0.000000
26179,89498c9addbffff,"POLYGON ((-103.08306 20.63489, -103.08184 20.6...",699.545645
26180,89498c98e43ffff,"POLYGON ((-103.00988 20.69909, -103.00866 20.7...",0.000000
26181,89498c9968bffff,"POLYGON ((-102.91380 20.64631, -102.91257 20.6...",0.000000


In [44]:
hex_gdf.loc[hex_gdf.time_museo>0].to_file(f'/home/edgar/SIG/ObsCds/Proyectos/Museo_CienciasAmbientales/Medio_Urbano/Analisis/hex_{res}_time_AMG_NewConect_v2.geojson', driver='GeoJSON')